In [1]:
import sklearn
from sklearnex import patch_sklearn
patch_sklearn()
#unpatch_sklearn()
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pandas import MultiIndex, Int16Dtype 
import xgboost as xgb
from xgboost import XGBRegressor
import daal4py as d4p
import joblib
import numpy as np
from time import perf_counter

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## About the Data

Engine degradation simulation was carried out using the Commercial Modular Aero-Propulsion System Simulation (C-MAPSS). Four different sets were simulated under different combinations of operational conditions and fault modes. This records several sensor channels to characterize fault evolution. The data set was provided by the NASA Ames Prognostics Center of Excellence (PCoE).

Data sets consists of multiple multivariate time series. Each data set is further divided into training and test subsets. Each time series is from a different engine i.e., the data can be considered to be from a fleet of engines of the same type. Each engine starts with different degrees of initial wear and manufacturing variation which is unknown to the user. This wear and variation is considered normal, i.e., it is not considered a fault condition. There are three operational settings that have a substantial effect on engine performance. These settings are also included in the data. The data is contaminated with sensor noise.

The engine is operating normally at the start of each time series, and develops a fault at some point during the series. In the training set, the fault grows in magnitude until system failure. In the test set, the time series ends some time prior to system failure. The objective of the competition is to predict the number of remaining operational cycles before failure in the test set, i.e., the number of operational cycles after the last cycle that the engine will continue to operate. Also provided a vector of true Remaining Useful Life (RUL) values for the test data.

The data are provided as a text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. The columns correspond to:

1) unit number
2) time, in cycles
3) operational setting 1
4) operational setting 2
5) operational setting 3
6) sensor measurement 1
7) sensor measurement 2
...
26) sensor measurement 26

Data Set: **FD001**
Train trjectories: 100
Test trajectories: 100
Conditions: ONE (Sea Level)
Fault Modes: ONE (HPC Degradation)

Data Set: **FD002**
Train trjectories: 260
Test trajectories: 259
Conditions: SIX
Fault Modes: ONE (HPC Degradation)

Data Set: **FD003**
Train trjectories: 100
Test trajectories: 100
Conditions: ONE (Sea Level)
Fault Modes: TWO (HPC Degradation, Fan Degradation)

Data Set: **FD004**
Train trjectories: 248
Test trajectories: 249
Conditions: SIX
Fault Modes: TWO (HPC Degradation, Fan Degradation)

**Reference**: 
[A. Saxena, K. Goebel, D. Simon, and N. Eklund, ‘Damage Propagation Modeling for Aircraft Engine Run-to-Failure Simulation’, in the Proceedings of the 1st International Conference on Prognostics and Health Management (PHM08), Denver CO, Oct 2008.](https://ntrs.nasa.gov/api/citations/20090029214/downloads/20090029214.pdf)

[Data Source 1](https://data.nasa.gov/Aerospace/CMAPSS-Jet-Engine-Simulated-Data/ff5v-kuh6)
[Data Source 2](https://www.nasa.gov/intelligent-systems-division/discovery-and-systems-health/pcoe/pcoe-data-set-repository/)

By changing fault modes and conditions, four different tests were conducted (thus, FD001, FD002, FD003, and FD004). For each test type, data were collected for training and test set. Training set data contains a number of engines being run simultaneously until it fails, i.e., for each engine data were collected to the point it failed.

While the experiments were conducted till failure, all data till failure are not made public. Instead, data for an arbitrary number of cycles are provided for each engine of the test set. The task is then to predict after how many cycles, each engine will fail. This predicted value is our remaining useful life (RUL). Our goal is to predict RUL as accurately as possible from operational data. That's why we call it data-driven RUL prediction.

In [2]:
train_data1 = pd.read_csv("../_Project_Innovator1_TurbofanEngineDegradationRUL/data/train_FD001.txt", sep = "\s+", header = None)
print(train_data1.shape)
print("Engines: ",np.unique(train_data1[0]))

(20631, 26)
Engines:  [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100]


In [3]:
train_data2 = pd.read_csv("../_Project_Innovator1_TurbofanEngineDegradationRUL/data/train_FD002.txt", sep = "\s+", header = None)
print(train_data2.shape)
print("Engines: ",np.unique(train_data2[0]))

(53759, 26)
Engines:  [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 

In [4]:
train_data3 = pd.read_csv("../_Project_Innovator1_TurbofanEngineDegradationRUL/data/train_FD003.txt", sep = "\s+", header = None)
print(train_data3.shape)
print("Engines: ",np.unique(train_data3[0]))

(24720, 26)
Engines:  [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100]


In [5]:
train_data4 = pd.read_csv("../_Project_Innovator1_TurbofanEngineDegradationRUL/data/train_FD004.txt", sep = "\s+", header = None)
print(train_data4.shape)
print("Engines: ",np.unique(train_data4[0]))

(61249, 26)
Engines:  [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 

This dataset has 26 columns (In fact, all 4 training and test datasets have 26 columns). Because of Python's numbering convention, the columns are numbered from 0 to 25. Column 1 is numbered as 0, Column 2 is numbered as 1, and so on. Description of each column is as follows:

Column 1: Corresponds to engine number (This column is indexed 0 above because of Python's numbering convention)
Column 2: Corresponds to cycle number. If engine 1 fails after 192 cycles, the entries of second column for engine 1 will go from 1 to 192. Similarly for other engines.
Columns 3,4,5: 3 operational settings
Columns 6-26: 21 sensor measurements

In [6]:
rul_data1 = pd.read_csv("../_Project_Innovator1_TurbofanEngineDegradationRUL/data/RUL_FD001.txt", sep = "\s+", header = None)

(100, 1)
     0
0  112
1   98
2   69
3   82
4   91
      0
95  137
96   82
97   59
98  117
99   20
98


In [7]:
rul_data2 = pd.read_csv("../_Project_Innovator1_TurbofanEngineDegradationRUL/data/RUL_FD002.txt", sep = "\s+", header = None)

(259, 1)
     0
0   18
1   79
2  106
3  110
4   15
       0
254  122
255  191
256   56
257  131
258   51
79


In [8]:
rul_data3 = pd.read_csv("../_Project_Innovator1_TurbofanEngineDegradationRUL/data/RUL_FD003.txt", sep = "\s+", header = None)

(100, 1)
     0
0   44
1   51
2   27
3  120
4  101
      0
95  113
96  123
97   17
98    8
99   28
51


In [9]:
rul_data4 = pd.read_csv("../_Project_Innovator1_TurbofanEngineDegradationRUL/data/RUL_FD004.txt", sep = "\s+", header = None)

(248, 1)
     0
0   22
1   39
2  107
3   75
4  149
       0
243   35
244  131
245  194
246  112
247   26
39


In [11]:
indices = rul_data1.index.tolist()
rul_dict1 = {index + 1: rul_data1.loc[index, 0] for index in indices}
train_data1['rul'] = train_data1[0].map(rul_dict1)

         0    1       2       3      4       5       6        7        8  \
20626  100  196 -0.0004 -0.0003  100.0  518.67  643.49  1597.98  1428.63   
20627  100  197 -0.0016 -0.0005  100.0  518.67  643.54  1604.50  1433.58   
20628  100  198  0.0004  0.0000  100.0  518.67  643.42  1602.46  1428.18   
20629  100  199 -0.0011  0.0003  100.0  518.67  643.23  1605.26  1426.53   
20630  100  200 -0.0032 -0.0005  100.0  518.67  643.85  1600.38  1432.14   

           9  ...       17       18      19    20   21    22     23     24  \
20626  14.62  ...  2388.26  8137.60  8.4956  0.03  397  2388  100.0  38.49   
20627  14.62  ...  2388.22  8136.50  8.5139  0.03  395  2388  100.0  38.30   
20628  14.62  ...  2388.24  8141.05  8.5646  0.03  398  2388  100.0  38.44   
20629  14.62  ...  2388.23  8139.29  8.5389  0.03  395  2388  100.0  38.29   
20630  14.62  ...  2388.26  8137.33  8.5036  0.03  396  2388  100.0  38.37   

            25  rul  
20626  22.9735   20  
20627  23.1594   20  
20628  2

In [12]:
indices2 = rul_data2.index.tolist()
rul_dict2 = {index + 1: rul_data2.loc[index, 0] for index in indices2}
train_data2['rul'] = train_data2[0].map(rul_dict2)
train_data2.drop(train_data2[train_data2[0] == 260].index, inplace=True)

{1: 18, 2: 79, 3: 106, 4: 110, 5: 15, 6: 155, 7: 6, 8: 90, 9: 11, 10: 79, 11: 6, 12: 73, 13: 30, 14: 11, 15: 37, 16: 67, 17: 68, 18: 99, 19: 22, 20: 54, 21: 97, 22: 10, 23: 142, 24: 77, 25: 88, 26: 163, 27: 126, 28: 138, 29: 83, 30: 78, 31: 75, 32: 11, 33: 53, 34: 173, 35: 63, 36: 100, 37: 151, 38: 55, 39: 48, 40: 37, 41: 44, 42: 27, 43: 18, 44: 6, 45: 15, 46: 112, 47: 131, 48: 13, 49: 122, 50: 13, 51: 98, 52: 53, 53: 52, 54: 106, 55: 103, 56: 152, 57: 123, 58: 26, 59: 178, 60: 73, 61: 169, 62: 39, 63: 39, 64: 14, 65: 11, 66: 121, 67: 86, 68: 56, 69: 115, 70: 17, 71: 148, 72: 104, 73: 78, 74: 86, 75: 98, 76: 36, 77: 94, 78: 52, 79: 91, 80: 15, 81: 141, 82: 74, 83: 146, 84: 17, 85: 47, 86: 194, 87: 21, 88: 79, 89: 97, 90: 8, 91: 9, 92: 73, 93: 183, 94: 97, 95: 73, 96: 49, 97: 31, 98: 97, 99: 9, 100: 14, 101: 106, 102: 8, 103: 8, 104: 106, 105: 116, 106: 120, 107: 61, 108: 168, 109: 35, 110: 80, 111: 9, 112: 50, 113: 151, 114: 78, 115: 91, 116: 7, 117: 181, 118: 150, 119: 106, 120: 15, 1

In [13]:
indices3 = rul_data3.index.tolist()
rul_dict3 = {index + 1: rul_data3.loc[index, 0] for index in indices3}
train_data3['rul'] = train_data3[0].map(rul_dict3)

         0    1       2       3      4       5       6        7        8  \
24715  100  148 -0.0016 -0.0003  100.0  518.67  643.78  1596.01  1424.11   
24716  100  149  0.0034 -0.0003  100.0  518.67  643.29  1596.38  1429.14   
24717  100  150 -0.0016  0.0004  100.0  518.67  643.84  1604.53  1431.41   
24718  100  151 -0.0023  0.0004  100.0  518.67  643.94  1597.56  1426.57   
24719  100  152  0.0000  0.0003  100.0  518.67  643.64  1599.04  1436.06   

           9  ...       17       18      19    20   21    22     23     24  \
24715  14.62  ...  2388.30  8138.08  8.5036  0.03  394  2388  100.0  38.44   
24716  14.62  ...  2388.28  8144.36  8.5174  0.03  395  2388  100.0  38.50   
24717  14.62  ...  2388.24  8135.95  8.5223  0.03  396  2388  100.0  38.39   
24718  14.62  ...  2388.26  8141.24  8.5148  0.03  395  2388  100.0  38.31   
24719  14.62  ...  2388.24  8136.98  8.5150  0.03  396  2388  100.0  38.56   

            25  rul  
24715  22.9631   28  
24716  22.9746   28  
24717  2

In [14]:
indices4 = rul_data4.index.tolist()
rul_dict4 = {index + 1: rul_data4.loc[index, 0] for index in indices4}
train_data4['rul'] = train_data4[0].map(rul_dict4)
train_data4.dropna(subset=['rul'], inplace=True)

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
rul    255
dtype: int64
         0    1        2       3      4       5       6        7        8  \
60989  248  180  35.0019  0.8409  100.0  449.44  556.28  1377.65  1148.96   
60990  248  181   0.0023  0.0000  100.0  518.67  643.95  1602.98  1429.57   
60991  248  182  25.0030  0.6200   60.0  462.54  536.88  1268.01  1067.09   
60992  248  183  41.9984  0.8414  100.0  445.00  550.64  1363.76  1145.72   
60993  248  184   0.0013  0.0001  100.0  518.67  643.50  1602.12  1430.34   

           9  ...       17       18       19    20   21    22      23     24  \
60989   5.48  ...  2387.77  8048.91   9.4169  0.02  337  2223  100.00  14.66   
60990  14.62  ...  2388.27  8122.44   8.5242  0.03  396  2388  100.

In [15]:
train_data = pd.concat([train_data1, train_data2, train_data3, train_data4], axis=0, ignore_index=True)
X = train_data.drop(['rul'], axis=1)
y = train_data['rul']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=123)

### XGBoost

Instantiate and define XGBoost regresion object by calling the XGBRegressor() class from the library. Use hyperparameters to define the object. Intel optimizations for XGBoost trainingcan be used by calling the hist tree method in the parameters, as shown below.

Fitting and training model using training datasets and predicting values. Note that Intel optimizations for XGBoost inference are enabled by default.

In [43]:
# Train the model
warnings.simplefilter(action='ignore', category=UserWarning)
t1_start = perf_counter()
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,max_depth = 30, alpha = 30, n_estimators = 200, tree_method='hist')
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
t1_stop = perf_counter()
print("Training time: {:.2f} seconds".format(t1_stop - t1_start))

Training time: 31.11 seconds


In [44]:
# Finding root mean squared error of predicted values.
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE:",rmse)

RMSE: 30.786741709841618


In [45]:
print("Here's our model:\n\n\n", model , "\n")
model_filename1 = '../_Project_Innovator1_TurbofanEngineDegradationRUL/models/xgb_regression.pkl'
# saving model to a file
joblib.dump(xg_reg, model_filename1) # nosec

Here's our model:


 NumberOfBetas: 27

NumberOfResponses: 1

InterceptFlag: False

Beta: array(
  [[ 0.00000000e+00  6.40636534e-02 -1.09749209e-02 -3.50389814e+01
     3.19743112e+02  6.98424277e+02  2.52246806e+02 -7.73440954e-01
     3.07479988e-02  1.76379633e-02 -1.36031299e+02  4.93662752e+00
    -3.04658466e-01  2.93264694e+00 -1.47145302e-03 -2.03652606e+01
    -1.73567696e+00 -3.24483302e-01  9.63718034e-01  2.15979649e-02
     8.30389231e+00  7.11025964e+00  2.62138032e-02 -1.05973137e+02
     4.55860894e+02  1.87719590e+00  3.67279622e-01]],
  dtype=float64, shape=(1, 27))

NumberOfFeatures: 26 



['../_Project_Innovator1_TurbofanEngineDegradationRUL/models/xgb_regression.pkl']

In [47]:
# loading the training model from a file
loaded_model1 = joblib.load(open(model_filename1, "rb")) # nosec
print("Here is one of our loaded model's features: \n\n", loaded_model1)

Here is one of our loaded model's features: 

 XGBRegressor(alpha=30, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=30, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=200, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)


https://stackabuse.com/bytes/how-to-save-and-load-xgboost-models/

In [48]:
preds = loaded_model1.predict(X_test)
# predicting the target feature(s) using the trained model

In [49]:
np.savetxt("../_Project_Innovator1_TurbofanEngineDegradationRUL/results/xgb_results.csv", preds, delimiter =  ",")
print("[CODE_SAMPLE_COMPLETED_SUCCESFULLY]")

[CODE_SAMPLE_COMPLETED_SUCCESFULLY]


In [21]:
# View the settings of the default XGBoost implementation.
model_xgb

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, disable_default_eval_metric='true',
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric='rmse', feature_types=None, gamma=None, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor='cpu_predictor', ...)

### DAAL4PY

In [22]:
# training the model for prediction
train_result = d4p.linear_regression_training().compute(X_train, y_train)

In [25]:
# retrieving and printing training model
model = train_result.model
print("Here's our model:\n\n\n", model , "\n")

model_filename = '../_Project_Innovator1_TurbofanEngineDegradationRUL/models/linear_regression_batch.pkl'

# saving model to a file
joblib.dump(model, model_filename) # nosec

Here's our model:


 NumberOfBetas: 27

NumberOfResponses: 1

InterceptFlag: False

Beta: array(
  [[ 0.00000000e+00  6.40636534e-02 -1.09749209e-02 -3.50389814e+01
     3.19743112e+02  6.98424277e+02  2.52246806e+02 -7.73440954e-01
     3.07479988e-02  1.76379633e-02 -1.36031299e+02  4.93662752e+00
    -3.04658466e-01  2.93264694e+00 -1.47145302e-03 -2.03652606e+01
    -1.73567696e+00 -3.24483302e-01  9.63718034e-01  2.15979649e-02
     8.30389231e+00  7.11025964e+00  2.62138032e-02 -1.05973137e+02
     4.55860894e+02  1.87719590e+00  3.67279622e-01]],
  dtype=float64, shape=(1, 27))

NumberOfFeatures: 26 



['../_Project_Innovator1_TurbofanEngineDegradationRUL/models/linear_regression_batch.pkl']

In [26]:
# loading the training model from a file
loaded_model = joblib.load(open(model_filename, "rb")) # nosec
print("Here is one of our loaded model's features: \n\n", loaded_model.Beta)

Here is one of our loaded model's features: 

 [[ 0.00000000e+00  6.40636534e-02 -1.09749209e-02 -3.50389814e+01
   3.19743112e+02  6.98424277e+02  2.52246806e+02 -7.73440954e-01
   3.07479988e-02  1.76379633e-02 -1.36031299e+02  4.93662752e+00
  -3.04658466e-01  2.93264694e+00 -1.47145302e-03 -2.03652606e+01
  -1.73567696e+00 -3.24483302e-01  9.63718034e-01  2.15979649e-02
   8.30389231e+00  7.11025964e+00  2.62138032e-02 -1.05973137e+02
   4.55860894e+02  1.87719590e+00  3.67279622e-01]]


In [27]:
# now predicting the target feature(s) using the trained model
y_pred = d4p.linear_regression_prediction().compute(X_test, loaded_model).prediction 

In [29]:
np.savetxt("../_Project_Innovator1_TurbofanEngineDegradationRUL/results/linear_regression_batch_results.csv", y_pred, delimiter =  ",")
print("[CODE_SAMPLE_COMPLETED_SUCCESFULLY]")

[CODE_SAMPLE_COMPLETED_SUCCESFULLY]
